### credits to "https://www.thecodeship.com/patterns/guide-to-python-function-decorators/"

In [ ]:
## The block below performs a decorating task on get_text() function using the func_wrapper() function

## The func_wrapper() gets an input argument which is the input required for get_text().

In [1]:
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)

def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

my_get_text = p_decorate(get_text)

print(my_get_text("John"))

<p>lorem ipsum, John dolor sit amet</p>


In [ ]:
## the same job as above block can be done using syntactic_sugar starting with @ 

In [16]:
def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

@p_decorate
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)


## the get_text() function is a decorated function from now on!

print(get_text("John"))

<p>lorem ipsum, John dolor sit amet</p>


# some more complex examples 

In [27]:
def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper

def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper

def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)


In [22]:
# getting a decorated of get_text() function nestedly entered into three decorating functions

# solution 1

In [28]:
get_text_decorated = div_decorate(p_decorate(strong_decorate(get_text)))
print(get_text_decorated('John'))

<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>


In [ ]:
# solution 2 

In [29]:
@div_decorate
@p_decorate
@strong_decorate
def get_text(name):
    return "lorem ipsum, {0} dolor sit amet".format(name)

print(get_text("John"))

## get_text() is now a decorated version of original get_text()! by calling get_text() we get the decorated version!

<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>


# Decorating Methods
# In Python, methods are functions that expect their first parameter to be a reference to the current object. We can build decorators for methods the same way, while taking self into consideration in the wrapper function.

In [2]:
def p_decorate(func):
    def func_wrapper(name):
        return "<p>{0}</p>".format(func(name))
    return func_wrapper

class Person(object):
    def __init__(self, in_name, in_family):
        self.name = in_name
        self.family = in_family

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

    
# instantiating the class        
my_person = Person('John', 'Doe')

# printing the output of get_fullname method in the class which is decorated by p_decorate
print(my_person.get_fullname())

# instantiating the class        
my_person = Person('Saeed', 'Mehrang')

# printing the output of get_fullname method in the class which is decorated by p_decorate
print(my_person.get_fullname())

<p>John Doe</p>
<p>Saeed Mehrang</p>


# A much better approach would be to make our decorator useful for functions and methods alike. This can be done by putting "args and *kwargs" as parameters for the wrapper, then it can accept any arbitrary number of arguments and keyword arguments.



In [3]:
def p_decorate(func):
    def func_wrapper(*args, **kwargs):
        return "<p>{0}</p>".format(func(*args, **kwargs))
    return func_wrapper

class Person(object):
    def __init__(self, in_name, in_family):
        self.name = in_name
        self.family = in_family

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

# instantiating the class        
my_person = Person('John', 'Doe')

# printing the output of get_fullname method in the class which is decorated by p_decorate
print(my_person.get_fullname())

# instantiating the class    
my_person = Person('Saeed', 'Mehrang')

# printing the output of get_fullname method in the class which is decorated by p_decorate
print(my_person.get_fullname())

<p>John Doe</p>
<p>Saeed Mehrang</p>


In [5]:

def p_decorate(func):
    def func_wrapper(*args, **kwargs): # replacing "*args, **kwargs" with a single variable such as "name" results in error 
        return "<p>{0}</p>".format(func(*args, **kwargs)) # replacing "*args, **kwargs" with a single variable such as "name" results in error
    return func_wrapper

class Person(object):
    def __init__(self, in_name, in_family):
        self.name = in_name
        self.family = in_family
        
    @p_decorate
    def get_fullname(self, nationality):
        return self.name+" "+self.family+" from "+ nationality

# instantiating the class    
my_person = Person('Saeed', 'Mehrang')

# printing the output of get_fullname method in the class which is decorated by p_decorate
print(my_person.get_fullname('Iran'))


<p>Saeed Mehrang from Iran</p>


# Passing arguments to decorators

In [6]:
def tags(tag_name):
    def tags_decorator(func):
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return func_wrapper
    return tags_decorator

@tags("p")
def get_text(name):
    return "Hello "+name

print(get_text("John"))

<p>Hello John</p>


 in the example above, the decorator gets the tag "p" and puts it in the tag_name. Next, the decorator reaches 
 the second line containing the tags_decorator(func) function. At this line, the decorator stops and waits for a 
 func to be defined and be entered into the place

# Debugging decorated functions

At the end of the day decorators are just wrapping our functions, in case of debugging that can be problematic since the wrapper function does not carry the name, module and docstring of the original function. Based on the example above if we do:

print(get_text.__name__)

The output was expected to be get_text yet, the attributes __name__, __doc__, and __module__ of get_text got overridden by those of the wrapper(func_wrapper). Obviously we can reset them within func_wrapper but Python provides a much nicer way.

Functools to the rescue
Fortunately Python (as of version 2.5) includes the functools module which contains functools.wraps. Wraps is a decorator for updating the attributes of the wrapping function(func_wrapper) to those of the original function(get_text). This is as simple as decorating func_wrapper by @wraps(func). Here is the updated example:

In [42]:
from functools import wraps

def tags(tag_name):
    def tags_decorator(func):
        @wraps(func)
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return func_wrapper
    return tags_decorator

@tags("p")
def get_text(name):
    """returns the doc string of the function"""
    return "Hello "+name

print(get_text.__name__) # get_text
print(get_text.__doc__) # returns some text
print(get_text.__module__) # __main__

get_text
returns the doc string of the function
__main__


In [43]:
get_text('john')

'<p>Hello john</p>'